# Imports

In [5]:
!pip install pandas
import pandas as pd
files=os.listdir('data/market_data')
tables=[]
for f in files:
    table=pd.read_csv(f)
    table.index=pd.todatetume(table.index)

'pip' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


ModuleNotFoundError: No module named 'pandas'

In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pickle
import os
import tensorflow as tf
from trading_calendars import get_calendar
import datetime

from trader.fetchers.Fetcher import Fetcher
from trader.Data import Data
from trader.Learning import Models,Metrics
from trader.Optimization import Optimization
from trader.Trade import Trade

import warnings
warnings.filterwarnings('ignore')

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Params

In [2]:
config = open("config.json") 
config = json.load(config) 
config

{'tickers_path': 'trader/data/tickers/tickers.csv',
 'tickers_conv_path': 'trader/data/tickers/to_yahoo_api_tickers_dict.p',
 'market_data_path': 'trader/data/market_data/',
 'news_data_path': 'trader/data/news_data/',
 'news_features_path': 'trader/data/news_data/news_features/',
 'merged_data_path': 'trader/data/merged_data/merged_data.p',
 'encoder_decoder_path': 'trader/models/encoder_decoder/',
 'bnn_path': 'trader/models/bnn/',
 'finbert_path': 'trader/models/sentiment_analysis/',
 'portfolios_path': 'trader/data/portfolios/',
 'predictions_path': 'trader/data/predictions/',
 'dropout_proba': 0.05,
 'sequence_length': 5,
 'dec_seq_length': 3,
 'latent_dim': 128,
 'api_key_id': 'PKUJ3Q7GC0IXU7RM6IQM',
 'secret_key': 'vFVk57T9JSsRCkY3ga9uCBC0E7BXwVsYERYUC5mE',
 'update_news': 1,
 'update_news_features': 1,
 'update_market': 1,
 'N_assets': 50,
 'C': 50,
 'Q': 0.1,
 'Q_rfa': 0,
 'num_of_generations': 500,
 'num_of_individuals': 200,
 'num_of_tour_particips': 10,
 'tournament_prob': 

In [3]:
us_calendar = get_calendar(config["calendar"])
r_f=config['r_f']

run_day=pd.to_datetime(config['run_day'],utc=True)+datetime.timedelta(hours=22)
pick_up_day=None
if(config['pick_up_day']!='None'):
    pick_up_day=pd.to_datetime(config['pick_up_day'],utc=True)+datetime.timedelta(hours=22)
    
previous_close=us_calendar.previous_close(pd.to_datetime(config['run_day'],utc=True))
delta=datetime.timedelta(hours=22-previous_close.hour-previous_close.minute/60)
   
#Defining training time and prediction time
train_time=us_calendar.previous_close(pd.to_datetime(config['run_day'],utc=True))+delta #22:00 UTC time
prediction_time=us_calendar.next_open(pd.to_datetime(config['run_day'],utc=True))

# To execute at train_time (collect data and train)

In [10]:
fetcher=Fetcher(tickers_path=config['tickers_path'],
    news_save_path=config['news_data_path'],
    market_save_path=config['market_data_path'],
    news_features_save_path=config['news_features_path'],
    finbert_path=config['finbert_path'],
    merged_data_path=config['merged_data_path'],
    index=config['market_index'],
    lookback=24+12+2,
    merging_time=22,
    #update_news=config['update_news'],
    update_news=0,
    #update_market=config['update_market'],
    update_market=0,
    #update_news_features=config['update_news_features'],
    update_news_features=0)

#fetcher.get_data();

In [9]:
print('Loading data')
data_object=Data(config['merged_data_path'],config['N_assets'],config['tickers_path'],config['market_index']);
data_object.load_data();
data_object.technical_analysis(data_object.F1,sequence_length=config['sequence_length']);
data_object.generate_sequences(data_object.F,run_day-np.timedelta64(24+12,'M'),run_day-np.timedelta64(12,'M'),
                               run_day,sequence_length=config['sequence_length'],
                               dec_seq_length=config['dec_seq_length'],latent_dim=config['latent_dim']);

print('Scaling external features')
scaler=MinMaxScaler()
data_object.F_train=scaler.fit_transform(data_object.F_train)
data_object.F_val=scaler.transform(data_object.F_val)
#data_object.F_test=scaler.transform(data_object.F_test)

print('loading models object')
models=Models(data_object.N,
    config['sequence_length'],
    config['dec_seq_length'],
    config['latent_dim']);

print('Encoder_decoder')

if (os.path.exists(config['encoder_decoder_path']+'encoder_'+str(run_day)+'.h5')):
    encoder_decoder=models.load_encoder_decoder(config['encoder_decoder_path']+'encoder_'+str(run_day)+'.h5')

else:
    if (pick_up_day==None):
        encoder_decoder=models.load_encoder_decoder()  
    else:
        encoder_decoder=models.load_encoder_decoder(config['encoder_decoder_path']+'encoder_'+str(pick_up_day)+'.h5')

    print('Training encoder_decoder')
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',restore_best_weights=True,patience=30)
    encoder_decoder.fit([data_object.X_train_enc, data_object.X_train_dec_in],data_object.X_train_dec,batch_size=2**4, epochs=50000, shuffle=True,
                        validation_data=[[data_object.X_val_enc, data_object.X_val_dec_in],data_object.X_val_dec],callbacks=[es])

print('Saving weights of encoder_decoder')
encoder_decoder.save_weights(config['encoder_decoder_path']+'encoder_'+str(run_day)+'.h5')
models.update_encoder_decoder(encoder_decoder)
encoder=models.get_encoder()

print('Encoding')
XX_val=np.asarray(encoder.predict(data_object.X_val))
XX_train=np.asarray(encoder.predict(data_object.X_train))
#XX_test=np.asarray(encoder.predict(data_object.X_test))

print('BNN')

if (os.path.exists(config['bnn_path']+'model_'+str(run_day)+'.h5')):
    bnn=models.load_BNN(XX_train,data_object.F_train,p=config['dropout_proba'],weights=config['bnn_path']+'model_'+str(run_day)+'.h5')

else:
    if (pick_up_day==None):
        bnn=models.load_BNN(XX_train,data_object.F_train,p=config['dropout_proba'])
    else :
        bnn=models.load_BNN(XX_train,data_object.F_train,p=config['dropout_proba'],weights=config['bnn_path']+'model_'+str(pick_up_day)+'.h5')

    print('Training BNN')
    metrics=Metrics(data_object.F_val,XX_val,data_object.Y_val,data_object.last_price_val,data_object.N,data_object.price_scalers,500,path=config['bnn_path']+'model_'+str(run_day)+'.h5')
    bnn.fit([data_object.F_train,XX_train],data_object.Y_train, epochs=50000, batch_size=2**4, shuffle=True,callbacks=[metrics])

    
print('Saving Weights of BNN')
bnn.load_weights(config['bnn_path']+'model_'+str(run_day)+'.h5')
models.update_bnn(bnn)

  0%|          | 0/50 [00:00<?, ?it/s]

Loading data


 30%|███       | 15/50 [00:28<01:05,  1.86s/it]

19350/19350 [==============================] - 11s 552us/sample - loss: 0.0047 - mse: 0.0047 - val_loss: 0.0069 - val_mse: 0.0069
Epoch 6/50000
19350/19350 [==============================] - 11s 555us/sample - loss: 0.0044 - mse: 0.0044 - val_loss: 0.0064 - val_mse: 0.0064
Epoch 7/50000
19350/19350 [==============================] - 11s 550us/sample - loss: 0.0040 - mse: 0.0040 - val_loss: 0.0060 - val_mse: 0.0060
Epoch 8/50000
19350/19350 [==============================] - 11s 555us/sample - loss: 0.0037 - mse: 0.0037 - val_loss: 0.0055 - val_mse: 0.0055
Epoch 9/50000
19350/19350 [==============================] - 11s 549us/sample - loss: 0.0034 - mse: 0.0034 - val_loss: 0.0051 - val_mse: 0.0051
Epoch 10/50000
19350/19350 [==============================] - 11s 554us/sample - loss: 0.0032 - mse: 0.0032 - val_loss: 0.0048 - val_mse: 0.0048
Epoch 11/50000
19350/19350 [==============================] - 11s 550us/sample - loss: 0.0029 - mse: 0.0029 - val_loss: 0.0044 - val_mse: 0.0044
Epoc

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



20112/20250 [============================>.] - ETA: 0s - loss: 0.6422 
0.013787206652539888
best =  0.01609987662420142
20250/20250 [==============================] - 7s 348us/sample - loss: 0.6422
Epoch 3261/50000
20176/20250 [============================>.] - ETA: 0s - loss: 0.6421 
0.005943214570909327
best =  0.01609987662420142
20250/20250 [==============================] - 7s 346us/sample - loss: 0.6421
Epoch 3262/50000
20048/20250 [============================>.] - ETA: 0s - loss: 0.6419 
0.003931265900361987
best =  0.01609987662420142
20250/20250 [==============================] - 7s 345us/sample - loss: 0.6419
Epoch 3263/50000
20032/20250 [============================>.] - ETA: 0s - loss: 0.6418 
0.01077543430108923
best =  0.01609987662420142
20250/20250 [==============================] - 7s 345us/sample - loss: 0.6418
Epoch 3264/50000
20192/20250 [============================>.] - ETA: 0s - loss: 0.6417 
0.028391450660032645
best =  0.01609987662420142
20250/20250 [========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



19968/20250 [============================>.] - ETA: 0s - loss: 0.6267 
0.010381432221619693
best =  0.01609987662420142
20250/20250 [==============================] - 7s 345us/sample - loss: 0.6267
Epoch 3398/50000
20064/20250 [============================>.] - ETA: 0s - loss: 0.6266 
0.005456223833846797
best =  0.01609987662420142
20250/20250 [==============================] - 7s 347us/sample - loss: 0.6266
Epoch 3399/50000
20112/20250 [============================>.] - ETA: 0s - loss: 0.6265 
0.0012071128600964784
best =  0.01609987662420142
20250/20250 [==============================] - 7s 348us/sample - loss: 0.6265
Epoch 3400/50000
20144/20250 [============================>.] - ETA: 0s - loss: 0.6264 
0.012074493684099128
best =  0.01609987662420142
20250/20250 [==============================] - 7s 346us/sample - loss: 0.6264
Epoch 3401/50000
19984/20250 [============================>.] - ETA: 0s - loss: 0.6263 
0.021186166432408497
best =  0.01609987662420142
20250/20250 [======

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# To execute at prediction_time (collect open price, predict, optimize,trade)

In [10]:
print('Collecting opening price')
data_object.update_F(run_day)
data_object.generate_sequences(data_object.F,run_day-np.timedelta64(24+12,'M'),run_day-np.timedelta64(12,'M'),
                               run_day,sequence_length=config['sequence_length'],
                               dec_seq_length=config['dec_seq_length'],latent_dim=config['latent_dim'],only_test=True);

XX_test=np.asarray(encoder.predict(data_object.X_test))
data_object.F_test=scaler.transform(data_object.F_test)

print('Prediction')
f=models.get_f()
predictions=models.predict(f,[data_object.F_test,XX_test],data_object.price_scalers);

print('Saving Prediction')

predictions_array=[run_day,predictions]

with open(config['predictions_path']+'predictions_'+str(run_day)+'.p', 'wb') as f:
    pickle.dump(predictions_array, f)

print('Estimating Expected Returns')
exptected_returns=models.calculate_exptected_returns(predictions,data_object);
exptected_returns=np.append(exptected_returns,(r_f*0.01));

print('Optimization')
optimizer=Optimization(data_object,config['portfolios_path'])
optimizer.optimize(exptected_returns,run_day,C=config['C'],Q=config['Q'],
                   Q_rfa=config['Q_rfa'],num_of_generations=config['num_of_generations'],
                  num_of_individuals=config['num_of_individuals'], num_of_tour_particips=config['num_of_tour_particips'],
                   tournament_prob=config['tournament_prob'], p_c=config['p_c'],p_m=config['p_m'],
                   p_risked=config['p_risked'],full=config['full'],policy=config['policy']);

100%|██████████| 50/50 [00:00<00:00, 97.80it/s]


Prediction
Saving Prediction
Estimating Expected Returns
Optimization


In [11]:
trade=Trade('config.json')
with open('trader/data/portfolios/portfolio_'+str(run_day)+'.p','rb') as f:
    portfolio = pickle.load(f)
    
portfolio=portfolio.features
trade.adjust_portfolio(portfolio)

ordering -1.0 of GOOGL
ordering -5.0 of JPM
ordering -5.0 of VZ
ordering -8.0 of BAC
ordering -15.0 of XOM
ordering -2.0 of CMCSA
ordering -1.0 of NVDA
ordering -1.0 of NKE
ordering -7.0 of BMY
ordering -4.0 of NEE
ordering 4.0 of AAPL
ordering 4.0 of MSFT
ordering 1.0 of FB
ordering 2.0 of V
ordering 7.0 of JNJ
ordering 3.0 of WMT
ordering 9.0 of PG
ordering 3.0 of MA
ordering 2.0 of UNH
ordering 13.0 of INTC
ordering 17.0 of T
ordering 1.0 of HD
ordering 17.0 of KO
ordering 12.0 of MRK
ordering 18.0 of PFE
ordering 1.0 of DIS
ordering 3.0 of PEP
ordering 2.0 of CSCO
ordering 15.0 of ORCL
ordering 5.0 of CVX
ordering 2.0 of ADBE
ordering 6.0 of ABT
ordering 5.0 of LLY
ordering 3.0 of CRM
ordering 6.0 of MCD
ordering 19.0 of WFC
ordering 3.0 of COST
ordering 6.0 of AMGN
ordering 2.0 of TMO
ordering 2.0 of PYPL
ordering 6.0 of ABBV
ordering 2.0 of PM
ordering 5.0 of AMT
ordering 8.0 of IBM
ordering 1.0 of LMT
ordering 1.0 of TSLA
ordering 3.0 of UNP
ordering 4.0 of HON
New portfolio :  